# Atividade 3 - Case
  
#### Conforme planilha de dados fornecida. 
Deseja-se prospectar empresas que possuam soluções em **tratamento de água** , 
principalmente,  elativas à : **solutions on waste and water, Improve water quality and
water efficiency use, water contamination, water for human consumption, water resources** .

**Foram identificados problemas no arquivo excel, com todos os dados localizados
na primeira coluna. Mesmo assim existiam dados onde não existia o fechamento das
aspas duplas, o número de atributos não estava completo, ou o atributo descritivo
possuía caracteres problemáticos. Devido a isto foi necessária a implementação da
carga, identificando e filtrando os problemas. Devido a isso se perderam 1988 
registros, os quais podem ser observados no arquivo 'error_log.json', isto nos
deixou 19309 de registros para trabalhar.**

In [1]:
import csv
import shlex
import re
from pathlib import Path
import json
import pandas as pd



In [2]:
ROOT_DIR = Path().absolute()
SOURCE_SUBDIR = 'Data'
DEST_SUBDIR = 'Data'
DATA_FILE_NAME = 'Pablo Vigneaux - Planilha em First_Desafio_ciencia_de_dados_SR.xlsm'
FULL_DATA_FILE_NAME = ROOT_DIR / SOURCE_SUBDIR / DATA_FILE_NAME


# Carregando os dados do Excel
df = pd.read_excel(FULL_DATA_FILE_NAME)

In [3]:
df.head(5)

,"name,description,employees,total_funding,city,subcountry,lat,lng",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,"fitin,""fitin is the latest fit tech platform t...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"quinditech,""our services include independent &...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"botfirst,""chatlanding is a digital chatbot pla...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"bravver,ai-powered army of health professional...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"stylify,""stylify is a mobile platform that con...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Corrigir a Estutura dos dados, a planilha Excel possui todas os atributos na primeira coluna

In [4]:
nomes_das_colunas = df.columns[0].split(',')
ser_primeira_coluna = df.iloc[:, 0]

In [5]:
ser_primeira_coluna.to_csv(ROOT_DIR / SOURCE_SUBDIR / 'temp.csv',
                           index=False,
                           header=False,
                           encoding='utf-8',
                           quoting=csv.QUOTE_NONE,
                           escapechar=" ")

In [6]:
data = []
error_log = []
num_columns = 8
lines = []
with open(ROOT_DIR / SOURCE_SUBDIR / 'temp.csv', "r") as hfile:
    for index, line in enumerate(hfile):
        line = re.sub(' +', ' ', line)
        line = re.sub('\s,', ',', line)
        #line = re.sub(r'[\x00-\x1f]', '', line)
        lines.append(line)
        try:
            if ',"' in line:
                splitter = shlex.shlex(line)
                splitter.whitespace = ','
                splitter.whitespace_split = True
                row = [x.strip(',') for x in splitter]
            else:
                row = line.strip().split(',')    
            if len(row) != num_columns:
                error_log.append(f'Erro: Columns num error ({len(row)}); \n Index: {index}; \n Text: {row}')
                continue
            data.append(row)
        except Exception as error:
            error_log.append(f'Erro: {error} \n Index: {index} \n Text: {line}')

            

In [7]:
with open(ROOT_DIR / SOURCE_SUBDIR / 'error_log.json', 'w') as herrorlog:
    json.dump(error_log, herrorlog, indent=4)

In [8]:
with open(ROOT_DIR / SOURCE_SUBDIR / 'data.json', 'w') as hdata:
    json.dump(data, hdata, indent=4)

In [9]:
df = pd.DataFrame(data, columns = nomes_das_colunas) 

In [11]:
df.to_parquet(ROOT_DIR / SOURCE_SUBDIR / 'data.parquet')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19309 entries, 0 to 19308
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           19309 non-null  object
 1   description    19309 non-null  object
 2   employees      19309 non-null  object
 3   total_funding  19309 non-null  object
 4   city           19309 non-null  object
 5   subcountry     19309 non-null  object
 6   lat            19309 non-null  object
 7   lng            19309 non-null  object
dtypes: object(8)
memory usage: 1.2+ MB
